# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

**Question 1**: What is the sum of the outputs of the generator for limit = 5?


In [4]:
sum(square_root_generator(5))

8.382332347441762

**Question 2:** What is the 13th number yielded by the generator?


In [43]:
for item in square_root_generator(13):
  pass
print(item)


3.605551275463989


**Question 3: **Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.


In [15]:
import dlt
import duckdb

In [37]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')

In [38]:
info = pipeline.run(people_1, write_disposition='append', primary_key='_dlt_id', table_name='people_append')

In [39]:
info = pipeline.run(people_2, write_disposition='append', primary_key='_dlt_id', table_name='people_append')

In [40]:
conn = duckdb.connect(f'{pipeline.pipeline_name}.duckdb')
conn.sql(f'SET search_path="{pipeline.dataset_name}"')

In [41]:
display(conn.sql('SELECT * from people_append').df())

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707947302.6285727,sX0IpF+azmoAow,None
1,2,Person_2,27,City_A,1707947302.6285727,sZ/9+Fy66gkSVA,None
2,3,Person_3,28,City_A,1707947302.6285727,+M29qsYOeBSxQA,None
3,4,Person_4,29,City_A,1707947302.6285727,Yf3sDoLbup0rwg,None
4,5,Person_5,30,City_A,1707947302.6285727,UJelktTAwI3sfg,None
5,3,Person_3,33,City_B,1707947303.2999005,qPV929vIgz8rDg,Job_3
6,4,Person_4,34,City_B,1707947303.2999005,AFMPeImY9RqAgA,Job_4
7,5,Person_5,35,City_B,1707947303.2999005,sPmLQG+mNfbhvQ,Job_5
8,6,Person_6,36,City_B,1707947303.2999005,oBaBXUDDsskx6A,Job_6
9,7,Person_7,37,City_B,1707947303.2999005,JvoHcHG596StfA,Job_7


In [35]:
#conn.sql('TRUNCATE TABLE people_append')

In [42]:
display(conn.sql('SELECT SUM(age) from people_append').df())

,sum(age)
0,353.0


**Question 4: ** Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.


In [7]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')

In [8]:
info = pipeline.run(people_1, write_disposition='merge', primary_key='ID', table_name='people')

In [9]:
info = pipeline.run(people_2, write_disposition='merge', primary_key='ID', table_name='people')

In [10]:
conn = duckdb.connect(f'{pipeline.pipeline_name}.duckdb')

In [11]:
conn.sql(f'SET search_path="{pipeline.dataset_name}"')

In [12]:
display(conn.sql("show tables"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [13]:
display(conn.sql('SELECT * from people').df())

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707946762.6631398,Uoi6iALxWeWMvQ,None
1,1,Person_1,26,City_A,1707946762.6631398,tpRqdFrgPaaSHQ,None
2,5,Person_5,35,City_B,1707946763.932182,xxOvHgHl487SSQ,Job_5
3,4,Person_4,34,City_B,1707946763.932182,ZQcxxdWCzKUBjg,Job_4
4,3,Person_3,33,City_B,1707946763.932182,7QWvvIMTim97Mw,Job_3
5,6,Person_6,36,City_B,1707946763.932182,fxssOi83jHajJg,Job_6
6,8,Person_8,38,City_B,1707946763.932182,YQju4qFTvRIWIQ,Job_8
7,7,Person_7,37,City_B,1707946763.932182,x2z2ey3qA2Q6Pg,Job_7


In [14]:
display(conn.sql('SELECT SUM(age) from people').df())

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX